<a href="https://colab.research.google.com/github/GDharan10/Data_Preprocessing_Framework/blob/main/Data_Preprocessing_Framework_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Librarys**

In [6]:
import pandas as pd
import numpy as np

#Statistic
from scipy import stats

#Visualization
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns


# **DataFrame**

In [7]:
df = sns.load_dataset('tips')
df.to_csv('tips.csv', index=False)

In [8]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


# **DataFrame observation**


In [9]:
continuous_columns = ["total_bill", "tip"]
category_columns = ["sex",	"smoker",	"day", "time", "size"]

#Unsupervised

# **Data cleaning using pandas**


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [11]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [12]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

# **Hypothesis testing / Statistical analysis**


Continuous - Central Limit Theorem, 1-Tailed Test (one-sample t-test)

Continuous vs. Continuous - Correlation, 2-Tailed Test (two-sample t-test)

Continuous vs. Categorical - ANOVA (Analysis of Variance)

Categorical vs. Categorical - Chi-Square Test

In [ ]:
'''
             Correlation
for all numerical columns

           Central Limit Theorem
total_bill
tip

             1-Tailed Test
total_bill
tip

              2-Tailed Test
total_bill,	tip 	   - H0 is rejected

                ANOVA
total_bill, sex	    - H0 is rejected
total_bill, smoker   - H0 is accepted
total_bill, day     - H0 is rejected
total_bill, time	    - H0 is rejected
total_bill, size    - H0 is rejected

tip, sex,	          - H0 is accepted
tip, smoker          - H0 is accepted
tip, day            - H0 is accepted
tip, time	          - H0 is accepted
tip, size            - H0 is rejected

            Chi-Square Test
sex, smoker          - H0 is accepted
sex, day              - H0 is rejected
sex, time	          - H0 is rejected
sex, size           - H0 is accepted

day, time	        - H0 is rejected
day, size	         - H0 is rejected

time, size	         - H0 is rejected
'''

# Correlation

In [14]:
df.corr()

<ipython-input-14-2f6f6606aa2c>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,total_bill,tip,size
total_bill,1.000000,0.675734,0.598315
tip,0.675734,1.000000,0.489299
size,0.598315,0.489299,1.000000


# Central Limit Theorem

In [15]:
def centralLimitTheorem(continuous_column, sampling_rate=0.20):
  result = {}
  samplesize = int(sampling_rate * len(df[continuous_column[0]]))
  for column in continuous_column:
    population=df[column].values
    population_mean = population.mean()
    sample_mean=[]
    for i in range(40):
      sample=np.random.choice(population,samplesize)
      sample_mean.append(sample.mean())
    result[column] = f"population mean: {population_mean} and samples mean: {np.mean(sample_mean)}"
  return result

# 1-Tailed Test

In [16]:
def oneTailTest(continuous_column, sampling_rate=0.05):
  result = {}
  samplesize = int(sampling_rate * len(df[continuous_column[0]]))
  for column in continuous_column:
    H0_accepted = 0
    H0_rejected = 0
    for i in range(samplesize):
      sample=df[column].sample(frac=0.05)
      t_test,p_value=stats.ttest_1samp(sample,df[column].mean())
      if p_value > 0.5:
        H0_accepted += 1
      else:
        H0_rejected += 1


    if H0_accepted > H0_rejected:
      result[column] = {
              "H0_accepted": H0_accepted,
              "H0_rejected": H0_rejected,
              "Conclusion": "H0 is accepted, Ha is rejected, There is no significant effect"
          }
    else:
        result[column] = {
            "H0_accepted": H0_accepted,
            "H0_rejected": H0_rejected,
            "Conclusion": "H0 is rejected, Ha is accepted, There is a significant effect"
        }
  return result

# 2-Tailed Test

In [17]:
def twoTailTest(continuous_column, sampling_rate=0.05):
  result = {}
  for i in range(len(continuous_column) - 1):
    column_1 = continuous_column[i]
    for column_2 in continuous_column[i+1:]:
      H0_accepted = 0
      H0_rejected = 0
      for i in range(20):
        sample1 = df[column_1].sample(frac=0.04)
        sample2 = df[column_2].sample(frac=0.04)
        t_test, p_value = stats.ttest_ind(sample1, sample2)
        if p_value > 0.5:
          H0_accepted += 1
        else:
          H0_rejected += 1

      if H0_accepted > H0_rejected:
        result[column_1 +' & '+ column_2] = {
                "H0_accepted": H0_accepted,
                "H0_rejected": H0_rejected,
                "Conclusion": "H0 is accepted, Ha is rejected, There is no significant effect"
            }
      else:
          result[column_1 +' & '+ column_2] = {
              "H0_accepted": H0_accepted,
              "H0_rejected": H0_rejected,
              "Conclusion": "H0 is rejected, Ha is accepted, There is a significant effect"
          }
  return result

# Chi-Square Test

In [18]:
def chiSquareTest(category_column):
  result = {}
  for i in range(len(category_column) - 1):
    column_1 = category_column[i]
    for column_2 in category_column[i+1:]:
      data = pd.crosstab(df[column_1], df[column_2])
      observed_values = data.values
      chi2_stat, p_value, _, _= stats.chi2_contingency(observed_values)

      if p_value > 0.05:
        result[column_1 +" & "+ column_2] = "H0 is accepted, There is no relationship between two columns we're comparing"
      else:
        result[column_1 +" & "+ column_2] = "H0 is rejected, There is a relationship between two columns we're comparing"
  return result

# ANOVA

In [19]:
def anova(category_column, continuous_column):
  result = {}
  for category_column in category_columns:
    for continuous_column in continuous_columns:
      group = df[category_column].unique()
      data = {}
      for i in group:
        data[i]=df[continuous_column][df[category_column]==i]

      f_value, p_value = stats.f_oneway(*[data[i] for i in group])
      if p_value > 0.05:
        result[category_column +" & "+ continuous_column] = "H0 is accepted, There is a relationship between two columns we're comparing"
      else:
        result[category_column +" & "+ continuous_column] = "H0 is rejected, There is no relationship between two columns we're comparing"
  return result

# **Result Hypothesis**

In [20]:
def hypothesis_test(continuous_columns, category_columns, sampling_rate = 0.05):
  centralLimitTheorem_result = centralLimitTheorem(continuous_columns, sampling_rate)
  onetailtest_result = oneTailTest(continuous_columns, sampling_rate)
  twotailtest_result = twoTailTest(continuous_columns, sampling_rate)
  Chi_Square_Test_result = chiSquareTest(category_columns)
  anova_result = anova(category_columns, continuous_columns)

In [21]:
centralLimitTheorem_result = centralLimitTheorem(continuous_columns, 0.25)
print("Central Limit Theorem result :\n")
for column, value in centralLimitTheorem_result.items():
  print(f"{column} : {value} \n")

sampling_rate = 0.05
onetailtest_result = oneTailTest(continuous_columns, sampling_rate)
print("\n1 sample t_test :\n")
for column, value in onetailtest_result.items():
    print(f"{column}: H0 accepted {value['H0_accepted']} times, H0 rejected {value['H0_rejected']} times")
    print(value['Conclusion'],'\n')

twotailtest_result = twoTailTest(continuous_columns)
print("\n2 sample t_test :\n")
for column, value in twotailtest_result.items():
    print(f"{column}: H0 accepted {value['H0_accepted']} times, H0 rejected {value['H0_rejected']} times")
    print(value['Conclusion'],'\n')

Chi_Square_Test_result = chiSquareTest(category_columns)
print("\nChi Square Test result :\n")
for column, value in Chi_Square_Test_result.items():
  print(f"{column} : {value} \n")

anova_result = anova(category_columns, continuous_columns)
print("\nANOVA Test :\n")
for column, value in Chi_Square_Test_result.items():
  print(f"{column} : {value} \n")

Central Limit Theorem result :

total_bill : population mean: 19.78594262295082 and samples mean: 19.585975409836063 

tip : population mean: 2.99827868852459 and samples mean: 2.9966065573770493 


1 sample t_test :

total_bill: H0 accepted 4 times, H0 rejected 8 times
H0 is rejected, Ha is accepted, There is a significant effect 

tip: H0 accepted 7 times, H0 rejected 5 times
H0 is accepted, Ha is rejected, There is no significant effect 


2 sample t_test :

total_bill & tip: H0 accepted 0 times, H0 rejected 20 times
H0 is rejected, Ha is accepted, There is a significant effect 


Chi Square Test result :

sex & smoker : H0 is accepted, There is no relationship between two columns we're comparing 

sex & day : H0 is rejected, There is a relationship between two columns we're comparing 

sex & time : H0 is rejected, There is a relationship between two columns we're comparing 

sex & size : H0 is accepted, There is no relationship between two columns we're comparing 

smoker & day : H

# **Data Visualization**



Continuous - Histogram, Boxplot, Violin Plot

Continuous vs Continuous - [Scatter / 3D scatter plot]

Categorical vs Categorical - Stacked Bar Chart, [Heatmap / Correlation Matrix]

Continuous vs Categorical - Bar Plot, [pie chart / donut chart], Clustered Bar Chart, Line

One Continuous vs More Categorical - sunburst plot, [stacked bar / grouped bar]


More: count plot, mosaic bar

In [22]:
correlation_matrix = df.corr()
fig = px.imshow(correlation_matrix, color_continuous_scale='Viridis',
                title="Correlation Matrix")
fig.show()

<ipython-input-22-ebcbbf98ef30>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df.corr()


In [23]:
#total_bill,	tip
fig = px.scatter(data_frame=df, x="total_bill", y="tip", color="sex", trendline="ols",
                 title="Scatter Plot")
fig.show()


In [24]:
#total_bill, day
df = px.data.tips()
fig = px.bar(data_frame=df, x="day", y="total_bill", color="sex", barmode="group",
             title="Clustered Bar Chart")
fig.show()


In [25]:
#sex, smoker
fig = px.bar(data_frame=df, x="sex", y="smoker", color="sex", barmode="stack",
             title="Stacked Bar Chart")
fig.show()


In [26]:
#tip, day
fig = px.bar(df, x = 'day',
                 y = 'tip',

                 title = 'Day vs Tip'
                )
fig.show()

In [27]:
pivot_table = df.pivot_table(values='tip', index='sex', columns='day', aggfunc='count')

print(pivot_table)

day     Fri  Sat  Sun  Thur
sex                        
Female    9   28   18    32
Male     10   59   58    30


# **Preprocessing**

In [28]:
df.time.unique()

array(['Dinner', 'Lunch'], dtype=object)

In [29]:
# Replacing categorical values with numerical equivalents
df['sex'] = df['sex'].map({'Female': 101, 'Male': 102})
df['smoker'] = df['smoker'].map({'No': 201, 'Yes': 202})
df['day'] = df['day'].replace({'Sun': 301, 'Thur': 302, 'Fri': 303, 'Sat' : 304})
df['time'] = df['time'].replace({'Dinner': 401, 'Lunch': 402})
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,101,201,301,401,2
1,10.34,1.66,102,201,301,401,3
2,21.01,3.50,102,201,301,401,3
3,23.68,3.31,102,201,301,401,2
4,24.59,3.61,101,201,301,401,4
...,...,...,...,...,...,...,...
239,29.03,5.92,102,201,304,401,3
240,27.18,2.00,101,202,304,401,2
241,22.67,2.00,102,202,304,401,2
242,17.82,1.75,102,201,304,401,2


# **Machine Learning**

In [31]:
# 1 data availability
# 2 separating independent and dependent
# 3 identifying algorithms/Model
# 4 training
# 5 evaluation